In [1]:
import os
import io
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, Subset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Dataset

In [2]:
from find_neighbors import get_station_neighbors


class AQIDailyDataset(Dataset):
    def __init__(self, raw_file_path, fpca_file_path, site_id, mode='train', valid_threshold=3):
        super().__init__()
        # 載入資料
        raw_loaded = np.load(raw_file_path, allow_pickle=True)
        fpca_loaded = np.load(fpca_file_path, allow_pickle=True)
        
        self.raw_data = raw_loaded['data']      # (5, 77, Day, 24)
        self.stations = raw_loaded['stations']
        self.dates = raw_loaded['dates']
        #self.fpca_data = fpca_loaded['data']    # (77, Day, 24)
        
        # 目標測站
        st_idx = np.where(self.stations == site_id)[0][0]
        st_raw = self.raw_data[:, st_idx, :, :].transpose(1, 0, 2) # (Day, 5, 24)
        
        # u,v
        wd_rad = np.deg2rad(st_raw[:, 0, :])
        ws = st_raw[:, 1, :]
        u, v = -ws * np.sin(wd_rad), -ws * np.cos(wd_rad)

        # 輔助測站
        neighbor_dict, pos_dict = get_station_neighbors(k=30)
        
        # 輔助測站 (PM2.5 Only)
        nb_ids = neighbor_dict.get(site_id, [])
        nb_indices = [np.where(self.stations == nid)[0][0] for nid in nb_ids if nid in self.stations]
        nb_pm25 = self.raw_data[3, nb_indices, :, :].transpose(1, 0, 2) # (Day, num_nb, 24)
        # 輔助測站，相對位置資訊
        self.nb_pos = pos_dict.get(site_id, [])   # (num_nb, 3)
        print("self.nb_pos", self.nb_pos.shape)

        # x: (Day, 5 + max_k, 24)
        num_nb = len(nb_indices)
        num_days = st_raw.shape[0]
        self.data_all = np.full((num_days, 5+num_nb, 24), np.nan)   # (D, 5+num_nb, 24) 有nan
        self.data_all[:, 0, :] = u / 15.0
        self.data_all[:, 1, :] = v / 15.0
        self.data_all[:, 2, :] = st_raw[:, 2, :] / 40.0
        self.data_all[:, 3, :] = st_raw[:, 3, :] / 100.0 # Target PM2.5 Raw
        self.data_all[:, 4, :] = st_raw[:, 4, :] / 100.0
        if len(nb_indices) > 0:
            self.data_all[:, 5:5+num_nb, :] = nb_pm25 / 100.0

        # 時間座標處理
        dt_dates = pd.to_datetime(self.dates)
        self.day_coords = np.stack([
            np.where(dt_dates.weekday.values >= 5, 1.0, 0.0),
            np.sin(2 * np.pi * dt_dates.weekday.values / 7.0),
            np.cos(2 * np.pi * dt_dates.weekday.values / 7.0),
            np.sin(2 * np.pi * (dt_dates.month.values - 1) / 12.0),
            np.cos(2 * np.pi * (dt_dates.month.values - 1) / 12.0)
        ], axis=1) # (Day, 5)
        
        # 過濾有效索引
        years = dt_dates.year.values
        indices_pool = np.where(years < 2025)[0] if mode == 'train' else np.where(years == 2025)[0]
        self.valid_indices = [
            i for i in indices_pool if i+1 < num_days and 
            (~np.isnan(self.data_all[i, 3, :])).sum() > valid_threshold and
            (~np.isnan(self.data_all[i+1, 3, :])).sum() > valid_threshold
        ]

        print( f"data_tensor_shape: {self.data_all.shape}")
        print( f"# of aux station: {num_nb}")
        print( f"# of {mode} samples: {len(self.valid_indices)}")

    def __len__(self): return len(self.valid_indices)

    def __getitem__(self, idx):
        d_idx = self.valid_indices[idx]
        
        x_val = self.data_all[d_idx]
        x_mask = (~np.isnan(x_val)).astype(np.float32)
        
        y_raw = self.data_all[d_idx+1, 3, :] 
        y_raw_mask = (~np.isnan(y_raw)).astype(np.float32) 

        other_y = self.data_all[d_idx+1, [0,1,2,4], :] 
        other_mask = (~np.isnan(other_y)).astype(np.float32) 
        
        day_coord = self.day_coords[d_idx+1]
        
        return {
            'x': torch.tensor(np.nan_to_num(x_val), dtype=torch.float32),
            'x_mask': torch.tensor(x_mask, dtype=torch.float32),
            'y_raw': torch.tensor(np.nan_to_num(y_raw), dtype=torch.float32),
            'y_raw_mask': torch.tensor(y_raw_mask, dtype=torch.float32),
            'day_coord': torch.tensor(day_coord, dtype=torch.float32),
            'aux_pos': torch.tensor(self.nb_pos, dtype=torch.float32)  # 鄰居相對位置資訊
        }

# DataModule

In [3]:
class AQIDataModule(pl.LightningDataModule):
    def __init__(self, raw_file_path: str, fpca_file_path: str, site_id: int, batch_size: int, valid_threshold=0.8):
        super().__init__()
        self.raw_file_path = raw_file_path
        self.fpca_file_path = fpca_file_path
        self.site_id = site_id
        self.batch_size = batch_size
        self.valid_threshold = valid_threshold

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            # 建立 2017-2024 的完整訓練數據集
            full_train_dataset = AQIDailyDataset(
                raw_file_path=self.raw_file_path,
                fpca_file_path=self.fpca_file_path,
                site_id=self.site_id, 
                mode='train', 
                valid_threshold=self.valid_threshold
            )
            num_samples = len(full_train_dataset)

            train_size = int(num_samples * 0.8)
            self.train_dataset = Subset(full_train_dataset, range(0, train_size)) 
            self.val_dataset = Subset(full_train_dataset, range(train_size, num_samples))  
 
        if stage == "test" or stage is None:
            # 建立 2025 的測試數據集
            self.test_dataset = AQIDailyDataset(
                raw_file_path=self.raw_file_path,
                fpca_file_path=self.fpca_file_path,
                site_id=self.site_id, 
                mode='test', 
                valid_threshold=self.valid_threshold
            )

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

# Pipeline

In [4]:
class AQIModelModule(pl.LightningModule):

    def __init__(self, model, site_id, lr=1e-3):
        super().__init__()
        self.model = model
        self.lr = lr
        self.site_id = site_id

        self.test_outputs = []

        self.site_dict_CH, self.site_dict_EN = self._build_siteid_to_name_dict("空氣品質監測站基本資料.csv")
        self.site_name_CH = self.site_dict_CH[site_id]
        self.site_name_EN = self.site_dict_EN[site_id]

    def _build_siteid_to_name_dict(self, csv_path: str):
        df = pd.read_csv(csv_path)
        df["siteid"] = df["siteid"].astype(int)
        site_dict_CH = dict(zip(df["siteid"], df["sitename"]))
        site_dict_EN = dict(zip(df["siteid"], df["siteengname"]))
        return site_dict_CH, site_dict_EN

    def forward(self, x, x_mask, day_coord, aux_pos):
        return self.model(x, x_mask, day_coord, aux_pos)

    # =========================
    # Core Shared Logic
    # =========================
    def _common_step(self, batch, stage: str):
        x = batch['x']
        x_mask = batch['x_mask']
        y_raw = batch['y_raw']
        y_raw_mask = batch['y_raw_mask']
        day_coord = batch['day_coord']
        aux_pos = batch['aux_pos']

        y_hat = self(x, x_mask, day_coord, aux_pos)

        loss = (F.mse_loss(y_hat, y_raw, reduction='none') * y_raw_mask).sum() / (y_raw_mask.sum() + 1e-8)

        y_real = y_raw * 100.0
        y_hat = y_hat * 100.0

        mse = (F.mse_loss(y_hat, y_real, reduction='none') * y_raw_mask).sum() / (y_raw_mask.sum() + 1e-8)
        mae = (torch.abs(y_hat - y_real) * y_raw_mask).sum() / (y_raw_mask.sum() + 1e-8)

        if stage != "train":
            self.log_dict({f"{stage}/mse": mse, f"{stage}/mae": mae}, prog_bar=True)

        return {
            "loss": loss,
            "mse": mse,
            "mae": mae,
            "y_hat": y_hat,
            "y_real": y_real,
            "mask": y_raw_mask
        }

    # =========================
    # Lightning Hooks
    # =========================
    def training_step(self, batch, batch_idx):
        out = self._common_step(batch, "train")
        self.log("train/loss", out["loss"], prog_bar=True)
        return out["loss"]

    def validation_step(self, batch, batch_idx):
        self._common_step(batch, "val")

    def test_step(self, batch, batch_idx):
        out = self._common_step(batch, "test")

        self.test_outputs.append({
            'y_real': out["y_real"].cpu().numpy(),
            'y_hat': out["y_hat"].cpu().numpy(),
            'mask': out["mask"].cpu().numpy()
        })

    # =========================
    # Test Epoch End
    # =========================
    def on_test_epoch_end(self):

        all_y = np.concatenate([x['y_real'] for x in self.test_outputs], axis=0)
        all_y_hat = np.concatenate([x['y_hat'] for x in self.test_outputs], axis=0)
        all_mask = np.concatenate([x['mask'] for x in self.test_outputs], axis=0)

        flat_y = all_y[all_mask == 1]
        flat_y_hat = all_y_hat[all_mask == 1]

        overall_mse = mean_squared_error(flat_y, flat_y_hat)
        overall_mae = mean_absolute_error(flat_y, flat_y_hat)
        overall_r2 = r2_score(flat_y, flat_y_hat)

        hourly_r2 = []
        for h in range(24):
            h_y = all_y[:, h]
            h_y_hat = all_y_hat[:, h]
            h_mask = all_mask[:, h]

            valid_h_y = h_y[h_mask == 1]
            valid_h_y_hat = h_y_hat[h_mask == 1]

            hourly_r2.append(r2_score(valid_h_y, valid_h_y_hat) if len(valid_h_y) > 1 else 0)

        self._plot_hourly_r2(hourly_r2, overall_mse, overall_r2)

        self.log("test/mse", overall_mse)
        self.log("test/mae", overall_mae)
        self.log("test/r2", overall_r2)

        self.test_outputs.clear()

    # =========================
    # Visualization
    # =========================
    def _plot_hourly_r2(self, hourly_r2, overall_mse, overall_r2):

        plt.figure(figsize=(10, 6))
        plt.plot(range(24), hourly_r2, marker='o', linestyle='-', color='b')

        title_str = (
            f'Hourly $R^2$ of {self.site_name_EN} (Test Set 2025)\n'
            f'Overall MSE: {overall_mse:.2f}, Overall $R^2$: {overall_r2:.4f}'
        )

        plt.title(title_str, fontsize=14)
        plt.xlabel('Hour of Day')
        plt.ylabel('R^2 Score')
        plt.grid(True)
        plt.xticks(range(24))
        plt.ylim([-0.1, 1])

        os.makedirs(f'results/{result_folder}', exist_ok=True)
        plt.savefig(f'results/{result_folder}/hourly_r2_sid_{self.site_id}_{self.site_name_CH}.png')
        plt.close()

    # =========================
    # Optimizer
    # =========================
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)


# Main

In [ ]:
from iTransformer_Entmax import iTransformer_Entmax


raw_file_path = 'aqi_data.npz'
fpca_file_path = 'after_fpca_PM2.5.npz'

offline = False

batch_size = 1024
max_epochs = 300

wandb_project = "iTransformer"
subname = "date+auxst+pos+Entmax"
result_folder = f"{wandb_project}_{subname}"

pl.seed_everything(42)  # 固定種子

loaded = np.load(raw_file_path, allow_pickle=True)
all_stations = loaded['stations']

all_station_results = []

for sid in all_stations:

    # if sid != 77:
    #     continue

    print(f"\n{'='*30}\nStarting Station: {sid}\n{'='*30}")
    
    wandb_logger = WandbLogger(
        project=wandb_project,
        name=f"Station_{sid}/{subname}",
        save_dir="logs",
        offline=offline,
        group=f"Station_{sid}"
    )
    
    dm = AQIDataModule(
        raw_file_path=raw_file_path,
        fpca_file_path=fpca_file_path,
        site_id=sid,
        batch_size=batch_size
    )
    
    base_model = iTransformer_Entmax(
    # [U, V, Temp, PM2.5, Hum, 5+K, timestamp]
        lookback_len=24, 
        d_model=128, 
        n_heads=8,
        e_layers=3,
        dropout=0.2
    )
    
    model = AQIModelModule(
        model=base_model, 
        site_id=sid,  
        lr=1e-3
    )
    
    checkpoint_callback = ModelCheckpoint(
        monitor="val/mse",
        dirpath=f"checkpoints/{result_folder}/station_{sid}",
        filename=f"siteID_{sid}_best-model",
        save_top_k=1,
        mode="min",
    )
    
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="auto", 
        devices=1,
        logger=wandb_logger,
        callbacks=[checkpoint_callback],
        log_every_n_steps=10
    )
    
    trainer.fit(model, datamodule=dm)

    test_result = trainer.test(model, datamodule=dm, ckpt_path="best")


    if test_result and len(test_result) > 0:
            all_station_results.append({
                'sid': sid,
                'mse': test_result[0]["test/mse"],
                'mae': test_result[0]["test/mae"],
                'r2':  test_result[0]["test/r2"]
            })
    
    wandb_logger.experiment.finish()
    torch.cuda.empty_cache()


if all_station_results:
    sids = np.array([str(r['sid']) for r in all_station_results])
    mses = np.array([r['mse'] for r in all_station_results])
    maes = np.array([r['mae'] for r in all_station_results])
    r2s  = np.array([r['r2'] for r in all_station_results])

    avg_mse, avg_mae, avg_r2 = np.mean(mses), np.mean(maes), np.mean(r2s)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 14))
    fig.suptitle(f"Global Evaluation Report: 77 Stations (2025 Test Set)", fontsize=22, fontweight='bold')

    # --- A. 繪製 MSE (由小到大排序) ---
    idx_mse = np.argsort(mses)
    ax1.bar(sids[idx_mse], mses[idx_mse], color='salmon', edgecolor='darkred', alpha=0.8)
    ax1.axhline(avg_mse, color='blue', linestyle='--', linewidth=2, label=f'Grand Mean MSE: {avg_mse:.2f}')
    ax1.set_title("MSE per Station (Lower is Better)", fontsize=16)
    ax1.set_ylabel("MSE", fontsize=12)
    ax1.tick_params(axis='x', rotation=90, labelsize=9)
    ax1.legend()

    # --- B. 繪製 R2 (由大到小排序) ---
    idx_r2 = np.argsort(r2s)[::-1] # 降序排列
    ax2.bar(sids[idx_r2], r2s[idx_r2], color='skyblue', edgecolor='navy', alpha=0.8)
    ax2.axhline(avg_r2, color='red', linestyle='--', linewidth=2, label=f'Grand Mean R²: {avg_r2:.2f}')
    ax2.set_title("R² Score per Station (Higher is Better)", fontsize=16)
    ax2.set_ylabel("R² Score", fontsize=12)
    ax2.set_ylim(0, 1)
    ax2.tick_params(axis='x', rotation=90, labelsize=9)
    ax2.legend()

    stats_text = (f"Grand Mean Summary\n"
              f"{'='*22}\n"
              f"Avg MSE: {avg_mse:.4f}\n"
              f"Avg MAE: {avg_mae:.4f}\n"
              f"Avg R² : {avg_r2:.4f}")

    fig.text(
        0.02, 0.5, stats_text,
        fontsize=18, family='monospace',
        bbox=dict(boxstyle='round', facecolor='white', edgecolor='black', alpha=0.95, pad=0.6),
        verticalalignment='center'
    )
    plt.tight_layout(rect=[0.08, 0.03, 1, 0.95]) 
    
    os.makedirs(f'results/{result_folder}', exist_ok=True)
    save_path = f'results/{result_folder}/global_comprehensive_report.png'
    plt.savefig(save_path, dpi=300)
    plt.show()

    print(f"\n✅ 雙指標評估報告已儲存至: {save_path}")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision



Starting Station: 1


wandb: Currently logged in as: chekai_kao (chekai_kao-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[Info] 已排除資料表中最後一個測站不參與計算: 員林
self.nb_pos (32, 3)
data_tensor_shape: (3256, 37, 24)
# of aux station: 32
# of train samples: 2863


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                | Params | Mode 
------------------------------------------------------
0 | model | iTransformer_Entmax | 449 K  | train
------------------------------------------------------
449 K     Trainable params
0         Non-trainable params
449 K     Total params
1.797     Total estimated model params size (MB)
58        Modules in train mode
0         Modules in eval mode


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]